In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
from math import pow
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from bitstring import BitArray

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
secret_mes = bytes(input("Enter the message you want to hide. Only use ASCII characters"),"ascii")

## Encryption

In [ ]:
key_len = 16
#keygen
key = get_random_bytes(key_len)
cipher = AES.new(key, AES.MODE_GCM)
#encrypt
ciphertext, tag = cipher.encrypt_and_digest(secret_mes)
encode_this = BitArray(cipher.nonce + ciphertext + tag)

In [ ]:
encode_this

## Encode

In [ ]:
result = "Dear audience,"
current_tokens = tokenizer(result, return_tensors="pt")
#print(current_tokens.input_ids)
next_token_logits = model(**current_tokens).logits[:, -1, :]
filtered_next_token_logits = torch.softmax(top_k_top_p_filtering(next_token_logits, top_p=0.9)[0],0)
print(filtered_next_token_logits.shape)
l_index = 0
r_index = len(filtered_next_token_logits)
j=0
while len(filtered_next_token_logits) > 1:
    b = encode_this[j]

    acc = 0
    if not b:
        i = 0
        while acc<0.5:
            acc += filtered_next_token_logits[i]
            i += 1
        i -= 1
        
        over = acc-0.5
        filtered_next_token_logits[i] -= over
        r_index -= len(filtered_next_token_logits)-i
        filtered_next_token_logits = 2*filtered_next_token_logits[:i+1]
    else:
        i = -1
        while acc<0.5:
            acc += filtered_next_token_logits[i]
            i -= 1
        i += 1

        over = acc-0.5
        filtered_next_token_logits[i] -= over
        l_index += len(filtered_next_token_logits)+i-1
        filtered_next_token_logits = 2*filtered_next_token_logits[i:] # in the end replace this by a softmax, will be more stable
    if j>20:
        print(filtered_next_token_logits)
    print(float(acc),i,filtered_next_token_logits.shape,j)
    print("lr index", l_index, r_index)
    j+=1


In [ ]:
a = ["a","b"]
a[-1:]

In [ ]:
current_string = "Dear audience,"
previous_string = ""

i = 0
while i < len(encode_this):
    b = encode_this[i]
    print("Current string:", current_string)
    current_tokens = tokenizer(current_string, return_tensors="pt")
    print(current_tokens.input_ids)
    probs = torch.topk(nn.functional.softmax(model(**current_tokens).logits[:, -1, :], dim=-1),100)
    (indeces_0, indeces_1) = balance_probs(probs)

    if b:
        (indeces_right, indeces_wrong) = (indeces_1, indeces_0)
    else:
        (indeces_right, indeces_wrong) = (indeces_0, indeces_1)
    
    print("Don't use these tokens ", [tokenizer.decode(probs.indices[0,i]) for i in indeces_wrong])
    print("Prefer these tokens ", [tokenizer.decode(probs.indices[0,i]) for i in indeces_right])
    
    new_token_string = input("add another token")
    new_token = tokenizer(new_token_string).input_ids[0]

    print([probs.indices[0,i] for i in indeces_right], [probs.indices[0,i] for i in indeces_wrong])

    if new_token in [probs.indices[0,i] for i in indeces_right]:
        i += 1
        print(i)
    elif new_token in [probs.indices[0,i] for i in indeces_wrong]:
        print("token from the 'don't use set'")
        continue

    previous_string = current_string[:]
    current_string += new_token_string

## Decode

In [ ]:
in_string = "Dear audience, please do your own due and take"
seed_len = 3
in_tokens = tokenizer(in_string, return_tensors="pt").input_ids

In [ ]:
bs = []

for i in range(seed_len, in_tokens.size()[1]):
    current_tokens = in_tokens[:,:i]
    probs = torch.topk(nn.functional.softmax(model(current_tokens).logits[:,-1, :], dim=-1),8)
    indeces = balance_probs(probs)
    new_token = in_tokens[0,i]

    if new_token in [probs.indices[0,i] for i in indeces[0]]:
        bs.append(False)
    elif new_token in [probs.indices[0,i] for i in indeces[1]]:
        bs.append(True)

decrypt_this = BitArray(bs).tobytes()

## Decryption

In [ ]:
nonce = decrypt_this[:key_len]
ciphertext = decrypt_this[key_len:-key_len]
tag = decrypt_this[-key_len:]

In [ ]:
#decrypt
cipher = AES.new(key, AES.MODE_GCM, nonce)
cipher.decrypt_and_verify(ciphertext, tag)